In [1]:
import os
os.chdir('/home/smallyan/eval_agent')
print(f"Working directory: {os.getcwd()}")

Working directory: /home/smallyan/eval_agent


# Generalizability Evaluation for InterpDetect

This notebook evaluates the generalizability of findings in the InterpDetect repository.

## Overview
InterpDetect proposes a mechanistic interpretability-based hallucination detection method for RAG systems using:
- **External Context Score (ECS)**: Measures how much attention heads attend to retrieved context
- **Parametric Knowledge Score (PKS)**: Measures FFN layers' contribution to the vocabulary distribution (via Jensen-Shannon divergence)

The method was developed on **Qwen3-0.6B** model and evaluated on **GPT-4.1-mini** responses.

## Evaluation Checklist:
- **GT1**: Generalization to a New Model
- **GT2**: Generalization to New Data  
- **GT3**: Method / Specificity Generalizability

## Constraints:
- Up to 3 trial examples for GT1 and GT2
- Up to 3 similar tasks for GT3
- Models used must not appear in original paper
- Data used must not appear in original dataset

In [2]:
# Setup and imports
import torch
import json
import os
import numpy as np
import sys

# Check GPU availability
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"CUDA device: {torch.cuda.get_device_name(0)}")
    print(f"Number of GPUs: {torch.cuda.device_count()}")
    device = "cuda"
else:
    device = "cpu"
print(f"Using device: {device}")

repo_path = '/net/scratch2/smallyan/InterpDetect_eval'

CUDA available: True
CUDA device: NVIDIA A100 80GB PCIe
Number of GPUs: 1
Using device: cuda


In [3]:
# Load and examine the test data to understand the structure
test_data_path = os.path.join(repo_path, 'datasets/test/test_w_chunk_score_qwen06b.json')
with open(test_data_path, 'r') as f:
    test_data = json.load(f)
    
print(f"Number of test examples: {len(test_data)}")
print(f"\nKeys in each example: {list(test_data[0].keys())}")

# Examine scores structure
print("\n\nSample score structure:")
print(json.dumps(test_data[0]['scores'][0], indent=2)[:2000])

Number of test examples: 256

Keys in each example: ['id', 'question', 'documents', 'documents_sentences', 'prompt', 'prompt_spans', 'num_tokens', 'response', 'response_spans', 'labels', 'hallucinated_llama-4-maverick-17b-128e-instruct', 'hallucinated_gpt-oss-120b', 'labels_llama', 'labels_gpt', 'scores']


Sample score structure:
{
  "prompt_attention_score": {
    "(0, 0)": 0.641965389251709,
    "(0, 1)": 0.641965389251709,
    "(0, 2)": 0.641965389251709,
    "(0, 3)": 0.8941237926483154,
    "(0, 4)": 0.641965389251709,
    "(0, 5)": 0.641965389251709,
    "(0, 6)": 0.641965389251709,
    "(0, 7)": 0.8941237926483154,
    "(0, 8)": 0.641965389251709,
    "(0, 9)": 0.641965389251709,
    "(0, 10)": 0.8941237926483154,
    "(0, 11)": 0.7777364253997803,
    "(0, 12)": 0.641965389251709,
    "(0, 13)": 0.641965389251709,
    "(0, 14)": 0.641965389251709,
    "(0, 15)": 0.641965389251709,
    "(1, 0)": 0.7777364253997803,
    "(1, 1)": 0.7777364253997803,
    "(1, 2)": 0.7777364253997

In [4]:
# Load the trained classifier to understand the model
import pickle

model_path = os.path.join(repo_path, 'trained_models/model_SVC_3000.pickle')
with open(model_path, 'rb') as f:
    classifier_bundle = pickle.load(f)

print("Keys in classifier bundle:", classifier_bundle.keys())
print("\nClassifier type:", type(classifier_bundle['model']))
print("Scaler type:", type(classifier_bundle.get('scaler')))

# Check feature names/indices if available
if 'feature_names' in classifier_bundle:
    print(f"Number of features: {len(classifier_bundle['feature_names'])}")
    print(f"First 10 features: {classifier_bundle['feature_names'][:10]}")

/home/smallyan/.conda/envs/meta/lib/python3.11/site-packages/sklearn/base.py:376: InconsistentVersionWarning: Trying to unpickle estimator StandardScaler from version 1.7.1 when using version 1.5.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/home/smallyan/.conda/envs/meta/lib/python3.11/site-packages/sklearn/base.py:376: InconsistentVersionWarning: Trying to unpickle estimator Pipeline from version 1.7.1 when using version 1.5.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


/home/smallyan/.conda/envs/meta/lib/python3.11/site-packages/sklearn/base.py:376: InconsistentVersionWarning: Trying to unpickle estimator SVC from version 1.7.1 when using version 1.5.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


AttributeError: 'Pipeline' object has no attribute 'keys'

In [5]:
# It's a Pipeline object, let's examine it
print("Classifier bundle type:", type(classifier_bundle))
print("\nPipeline steps:", classifier_bundle.steps)

# Look at the SVC model's parameters
svc_model = classifier_bundle.named_steps.get('svc') or classifier_bundle.steps[-1][1]
print("\nSVC model:", svc_model)

Classifier bundle type: <class 'sklearn.pipeline.Pipeline'>

Pipeline steps: [('pipeline', Pipeline(steps=[('standardscaler', StandardScaler())])), ('svc', SVC())]

SVC model: SVC()


## Understanding the InterpDetect Method

Based on the repository analysis:

### Key Findings (Neuron-Level):
1. **External Context Score (ECS)**: Computed per attention head and layer by measuring cosine similarity between response and most-attended context chunk. Hallucinated responses show **lower** ECS (less external context utilization).

2. **Parametric Knowledge Score (PKS)**: Computed per FFN layer using Jensen-Shannon divergence between vocabulary distributions before and after FFN processing. Hallucinated responses show **higher** PKS in later layers (more parametric knowledge injection).

### Original Experimental Setup:
- **Model**: Qwen3-0.6B (28 layers, 16 attention heads)
- **Dataset**: RAGBench/FinQA 
- **Classifier**: SVC trained on 3000 samples using ECS and PKS features

### Evaluation Goals:
- **GT1**: Test on a NEW model (not Qwen3-0.6B or GPT-4.1-mini)
- **GT2**: Test on NEW data (not from original FinQA dataset)
- **GT3**: Test method on another similar task (if applicable)

---
# GT1: Model Generalization Evaluation

**Goal**: Test if the ECS/PKS-based hallucination detection method generalizes to a NEW model not used in the original work.

**Original Models Used**:
- Qwen3-0.6B (for computing ECS/PKS signals)
- GPT-4.1-mini (for response generation in proxy evaluation)

**New Model for Testing**: We will use **Phi-3-mini-4k-instruct** (3.8B parameters) from Microsoft, which is:
- A different architecture than Qwen
- Not mentioned in the original paper
- Available in TransformerLens

**Approach**: 
1. Load a different model (Phi-3-mini or similar)
2. Compute ECS/PKS scores on existing test data
3. Verify that the correlation patterns (ECS negative, PKS positive with hallucination) hold

In [6]:
# First, let's check what models are available in TransformerLens
from transformer_lens import HookedTransformer

# List some common models
print("Checking available models in TransformerLens...")
# We'll try to use a different model - let's try Llama-2 or Mistral or Gemma
# These are architecturally different from Qwen

# Let's see what models work
available_models = [
    "gpt2-small",
    "gpt2-medium",
    "pythia-70m",
    "pythia-160m",
    "pythia-410m",
    "pythia-1b",
    "gemma-2b",
    "llama-7b",
    "mistral-7b"
]
print("Models to try:", available_models)

Checking available models in TransformerLens...
Models to try: ['gpt2-small', 'gpt2-medium', 'pythia-70m', 'pythia-160m', 'pythia-410m', 'pythia-1b', 'gemma-2b', 'llama-7b', 'mistral-7b']


In [7]:
# Let's use Gemma-2B which is a different architecture from Qwen
# This is a good candidate as it's not mentioned in the original work
# and is a different model family

print("Loading Gemma-2B model for GT1 evaluation...")

try:
    new_model = HookedTransformer.from_pretrained(
        "gemma-2b",
        device="cpu",
        torch_dtype=torch.float16
    )
    new_model.to(device)
    print(f"Successfully loaded Gemma-2B!")
    print(f"Model config: {new_model.cfg.n_layers} layers, {new_model.cfg.n_heads} heads")
    MODEL_NAME = "gemma-2b"
except Exception as e:
    print(f"Failed to load Gemma-2B: {e}")
    print("Trying Pythia-1B instead...")
    try:
        new_model = HookedTransformer.from_pretrained(
            "pythia-1b",
            device="cpu",
            torch_dtype=torch.float16
        )
        new_model.to(device)
        print(f"Successfully loaded Pythia-1B!")
        print(f"Model config: {new_model.cfg.n_layers} layers, {new_model.cfg.n_heads} heads")
        MODEL_NAME = "pythia-1b"
    except Exception as e2:
        print(f"Failed to load Pythia-1B: {e2}")
        print("Trying GPT-2 medium...")
        new_model = HookedTransformer.from_pretrained(
            "gpt2-medium",
            device="cpu",
            torch_dtype=torch.float32
        )
        new_model.to(device)
        print(f"Successfully loaded GPT-2 medium!")
        print(f"Model config: {new_model.cfg.n_layers} layers, {new_model.cfg.n_heads} heads")
        MODEL_NAME = "gpt2-medium"

Loading Gemma-2B model for GT1 evaluation...
Failed to load Gemma-2B: gemma-2b not found. Valid official model names (excl aliases): ['gpt2', 'gpt2-medium', 'gpt2-large', 'gpt2-xl', 'distilgpt2', 'facebook/opt-125m', 'facebook/opt-1.3b', 'facebook/opt-2.7b', 'facebook/opt-6.7b', 'facebook/opt-13b', 'facebook/opt-30b', 'facebook/opt-66b', 'EleutherAI/gpt-neo-125M', 'EleutherAI/gpt-neo-1.3B', 'EleutherAI/gpt-neo-2.7B', 'EleutherAI/gpt-j-6B', 'EleutherAI/gpt-neox-20b', 'stanford-crfm/alias-gpt2-small-x21', 'stanford-crfm/battlestar-gpt2-small-x49', 'stanford-crfm/caprica-gpt2-small-x81', 'stanford-crfm/darkmatter-gpt2-small-x343', 'stanford-crfm/expanse-gpt2-small-x777', 'stanford-crfm/arwen-gpt2-medium-x21', 'stanford-crfm/beren-gpt2-medium-x49', 'stanford-crfm/celebrimbor-gpt2-medium-x81', 'stanford-crfm/durin-gpt2-medium-x343', 'stanford-crfm/eowyn-gpt2-medium-x777', 'EleutherAI/pythia-70m', 'EleutherAI/pythia-160m', 'EleutherAI/pythia-410m', 'EleutherAI/pythia-1b', 'EleutherAI/pythia-

Could not cache non-existence of file. Will ignore error and continue. Error: [Errno 122] Disk quota exceeded: '/net/projects/chai-lab/shared_models/hub/models--gpt2-medium/.no_exist/6dcaa7a952f72f9298047fd5137cd6e4f05f41da/adapter_config.json'


Could not cache non-existence of file. Will ignore error and continue. Error: [Errno 122] Disk quota exceeded: '/net/projects/chai-lab/shared_models/hub/models--gpt2-medium/.no_exist/6dcaa7a952f72f9298047fd5137cd6e4f05f41da/adapter_config.json'


TypeError: HookedTransformer.__init__() got an unexpected keyword argument 'torch_dtype'

In [8]:
# Let's try GPT-2 medium without the torch_dtype parameter
print("Loading GPT-2 medium model for GT1 evaluation...")

new_model = HookedTransformer.from_pretrained(
    "gpt2-medium",
    device="cpu"
)
new_model.to(device)
print(f"Successfully loaded GPT-2 medium!")
print(f"Model config: {new_model.cfg.n_layers} layers, {new_model.cfg.n_heads} heads")
MODEL_NAME = "gpt2-medium"

Loading GPT-2 medium model for GT1 evaluation...


Could not cache non-existence of file. Will ignore error and continue. Error: [Errno 122] Disk quota exceeded: '/net/projects/chai-lab/shared_models/hub/models--gpt2-medium/.no_exist/6dcaa7a952f72f9298047fd5137cd6e4f05f41da/adapter_config.json'


Could not cache non-existence of file. Will ignore error and continue. Error: [Errno 122] Disk quota exceeded: '/net/projects/chai-lab/shared_models/hub/models--gpt2-medium/.no_exist/6dcaa7a952f72f9298047fd5137cd6e4f05f41da/adapter_config.json'


Loaded pretrained model gpt2-medium into HookedTransformer
Moving model to device:  cuda


Successfully loaded GPT-2 medium!
Model config: 24 layers, 16 heads


In [9]:
# Load necessary libraries for computing ECS and PKS
from transformers import AutoTokenizer
from sentence_transformers import SentenceTransformer
from torch.nn import functional as F
import gc

# Load tokenizer and sentence transformer
tokenizer = AutoTokenizer.from_pretrained("gpt2-medium")
bge_model = SentenceTransformer("BAAI/bge-base-en-v1.5").to(device)

print("Loaded tokenizer and sentence transformer")
print(f"Max context length for GPT-2 medium: {new_model.cfg.n_ctx}")

No sentence-transformers model found with name BAAI/bge-base-en-v1.5. Creating a new one with mean pooling.


Could not cache non-existence of file. Will ignore error and continue. Error: [Errno 122] Disk quota exceeded: '/net/projects/chai-lab/shared_models/hub/models--BAAI--bge-base-en-v1.5'


OSError: [Errno 122] Disk quota exceeded: '/net/projects/chai-lab/shared_models/hub/models--BAAI--bge-base-en-v1.5'

In [10]:
# Let's set a local cache directory to avoid disk quota issues
import os
os.environ['HF_HOME'] = '/net/scratch2/smallyan/hf_cache'
os.environ['TRANSFORMERS_CACHE'] = '/net/scratch2/smallyan/hf_cache'
os.makedirs('/net/scratch2/smallyan/hf_cache', exist_ok=True)

# Try loading the sentence transformer again
from sentence_transformers import SentenceTransformer
bge_model = SentenceTransformer("BAAI/bge-base-en-v1.5").to(device)

print("Loaded sentence transformer successfully!")
print(f"Max context length for GPT-2 medium: {new_model.cfg.n_ctx}")

No sentence-transformers model found with name BAAI/bge-base-en-v1.5. Creating a new one with mean pooling.


Could not cache non-existence of file. Will ignore error and continue. Error: [Errno 122] Disk quota exceeded: '/net/projects/chai-lab/shared_models/hub/models--BAAI--bge-base-en-v1.5'


OSError: [Errno 122] Disk quota exceeded: '/net/projects/chai-lab/shared_models/hub/models--BAAI--bge-base-en-v1.5'

In [11]:
# Set environment variable before import to force using the scratch directory
import os
os.environ['HF_HOME'] = '/net/scratch2/smallyan/hf_cache'
os.environ['HF_HUB_CACHE'] = '/net/scratch2/smallyan/hf_cache'
os.environ['TRANSFORMERS_CACHE'] = '/net/scratch2/smallyan/hf_cache'
os.environ['HUGGINGFACE_HUB_CACHE'] = '/net/scratch2/smallyan/hf_cache'

# Create cache directory
os.makedirs('/net/scratch2/smallyan/hf_cache', exist_ok=True)

# Need to reload the module
import importlib
import huggingface_hub
importlib.reload(huggingface_hub)

from sentence_transformers import SentenceTransformer
bge_model = SentenceTransformer("BAAI/bge-base-en-v1.5", cache_folder='/net/scratch2/smallyan/hf_cache').to(device)

print("Loaded sentence transformer successfully!")
print(f"Max context length for GPT-2 medium: {new_model.cfg.n_ctx}")

Could not cache non-existence of file. Will ignore error and continue. Error: [Errno 122] Disk quota exceeded: '/net/projects/chai-lab/shared_models/hub/models--BAAI--bge-base-en-v1.5'


OSError: [Errno 122] Disk quota exceeded: '/net/projects/chai-lab/shared_models/hub/models--BAAI--bge-base-en-v1.5'

In [12]:
# The disk quota issue is preventing model downloads. Let's work around this
# by using simpler cosine similarity with the model's own embeddings

# For GT1 evaluation, we will compute:
# 1. PKS (parametric knowledge score) using Jensen-Shannon divergence - this doesn't require BGE
# 2. For ECS, we can use the model's own embeddings instead of BGE

# Let's implement a simplified version that verifies the core finding:
# - Higher PKS in later layers correlates with hallucination
# - Lower ECS correlates with hallucination

print("GT1 Evaluation Strategy:")
print("========================")
print("Since we have disk quota issues, we will:")
print("1. Use GPT-2 medium (different architecture from Qwen3-0.6B)")
print("2. Compute PKS (Jensen-Shannon divergence) which doesn't require external models")
print("3. Verify the correlation between PKS and hallucination labels")
print("\nThis tests whether the PKS-hallucination correlation generalizes to a new model.")
print(f"\nModel: {MODEL_NAME}")
print(f"Layers: {new_model.cfg.n_layers}")
print(f"Heads: {new_model.cfg.n_heads}")

GT1 Evaluation Strategy:
Since we have disk quota issues, we will:
1. Use GPT-2 medium (different architecture from Qwen3-0.6B)
2. Compute PKS (Jensen-Shannon divergence) which doesn't require external models
3. Verify the correlation between PKS and hallucination labels

This tests whether the PKS-hallucination correlation generalizes to a new model.

Model: gpt2-medium
Layers: 24
Heads: 16


In [13]:
# Load test data and prepare for GT1 evaluation
test_data_path = os.path.join(repo_path, 'datasets/test/test_w_chunk_score_qwen06b.json')
with open(test_data_path, 'r') as f:
    test_data = json.load(f)

print(f"Loaded {len(test_data)} test examples")

# Let's select 3 examples for trial (as per constraint)
# We'll pick examples that have clear hallucination labels
trial_examples = []
for ex in test_data[:20]:  # Look at first 20 to find good examples
    has_hallucination = any(score['hallucination_label'] == 1 for score in ex['scores'])
    has_truth = any(score['hallucination_label'] == 0 for score in ex['scores'])
    if has_hallucination and has_truth:
        trial_examples.append(ex)
        if len(trial_examples) >= 3:
            break

print(f"\nSelected {len(trial_examples)} trial examples with mixed hallucination labels")
for i, ex in enumerate(trial_examples):
    labels = [s['hallucination_label'] for s in ex['scores']]
    print(f"  Example {i+1}: {ex['id']}, labels: {labels}")

Loaded 256 test examples

Selected 3 trial examples with mixed hallucination labels
  Example 1: finqa_6345, labels: [0, 0, 1, 1, 0]
  Example 2: finqa_6634, labels: [0, 1, 0]
  Example 3: finqa_6587, labels: [0, 0, 0, 1, 1]


In [14]:
# Define PKS computation function (adapted from original code)
from torch.nn import functional as F

def calculate_js_divergence(x_mid, x_post, W_U):
    """Calculate Jensen-Shannon divergence between vocabulary distributions before and after FFN"""
    # Get vocabulary distributions
    sep_vocabulary_dist = x_mid @ W_U
    sep_attention_dist = x_post @ W_U
    
    # Calculate softmax
    softmax_mature_layer = F.softmax(sep_vocabulary_dist, dim=-1)
    softmax_anchor_layer = F.softmax(sep_attention_dist, dim=-1)

    # Calculate the average distribution M
    M = 0.5 * (softmax_mature_layer + softmax_anchor_layer)

    # Calculate log-softmax for the KL divergence
    log_softmax_mature_layer = F.log_softmax(sep_vocabulary_dist, dim=-1)
    log_softmax_anchor_layer = F.log_softmax(sep_attention_dist, dim=-1)

    # Calculate the KL divergences and then the JS divergences
    kl1 = F.kl_div(log_softmax_mature_layer, M, reduction='none').sum(dim=-1)
    kl2 = F.kl_div(log_softmax_anchor_layer, M, reduction='none').sum(dim=-1)
    js_divs = 0.5 * (kl1 + kl2)

    scores = js_divs.cpu().tolist()
    return sum(scores)

print("PKS computation function defined")

PKS computation function defined


In [15]:
# Load tokenizer for GPT-2 medium
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("gpt2-medium")

# GPT-2 doesn't have a pad token by default
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

print(f"Tokenizer loaded: {tokenizer.__class__.__name__}")
print(f"Vocab size: {tokenizer.vocab_size}")

Tokenizer loaded: GPT2TokenizerFast
Vocab size: 50257


In [16]:
# Compute PKS scores for trial examples on GPT-2 medium
import gc

def compute_pks_for_example(example, model, tokenizer, device, max_ctx=1024):
    """Compute PKS scores for all response spans in an example"""
    response = example['response']
    prompt = example['prompt']
    
    # Prepare input
    full_text = f"{prompt}\n{response}"
    input_ids = tokenizer(full_text, return_tensors="pt", truncation=True, max_length=max_ctx).input_ids.to(device)
    
    # Get prompt length for span calculation
    prompt_ids = tokenizer(prompt, return_tensors="pt").input_ids
    prefix_len = prompt_ids.shape[-1]
    
    # Run model with cache
    with torch.no_grad():
        logits, cache = model.run_with_cache(input_ids, return_type="logits")
    
    # Calculate PKS for each layer at response positions
    pks_by_layer = {}
    response_start = min(prefix_len, input_ids.shape[-1] - 1)
    response_end = input_ids.shape[-1]
    
    for layer_idx in range(model.cfg.n_layers):
        # Get residual stream before and after FFN (mid and post)
        x_mid = cache[f"blocks.{layer_idx}.hook_resid_mid"][0, response_start:response_end, :]
        x_post = cache[f"blocks.{layer_idx}.hook_resid_post"][0, response_start:response_end, :]
        
        # Compute PKS (JS divergence)
        pks = calculate_js_divergence(x_mid, x_post, model.W_U)
        pks_by_layer[f"layer_{layer_idx}"] = pks
    
    # Clean up
    del cache, logits
    gc.collect()
    torch.cuda.empty_cache()
    
    return pks_by_layer

# Test on first trial example
print("Computing PKS for trial example 1...")
trial_example = trial_examples[0]
pks_scores = compute_pks_for_example(trial_example, new_model, tokenizer, device)

print(f"\nPKS scores across {len(pks_scores)} layers:")
for layer, score in list(pks_scores.items())[:5]:
    print(f"  {layer}: {score:.4f}")
print("  ...")
for layer, score in list(pks_scores.items())[-3:]:
    print(f"  {layer}: {score:.4f}")

Computing PKS for trial example 1...



PKS scores across 24 layers:
  layer_0: 1244.9322
  layer_1: 90.9779
  layer_2: 42.7840
  layer_3: 44.0928
  layer_4: 59.5010
  ...
  layer_21: 80.6313
  layer_22: 57.1498
  layer_23: 77.4341


In [17]:
# Now we need to compute span-level PKS to compare with hallucination labels
# The original method computes PKS per response span

def compute_span_pks(example, model, tokenizer, device, max_ctx=1024):
    """Compute PKS scores for each response span in an example"""
    response = example['response']
    prompt = example['prompt']
    response_spans = example['response_spans']  # Character-level spans
    
    # Prepare input
    full_text = f"{prompt}\n{response}"
    input_ids = tokenizer(full_text, return_tensors="pt", truncation=True, max_length=max_ctx).input_ids.to(device)
    
    # Get prompt length in tokens
    prompt_tokens = tokenizer(prompt + "\n", return_tensors="pt").input_ids
    prefix_len = prompt_tokens.shape[-1]
    
    # Convert character spans to token spans
    token_spans = []
    for char_start, char_end in response_spans:
        # Get text up to span start/end
        text_to_start = prompt + "\n" + response[:char_start]
        text_to_end = prompt + "\n" + response[:char_end]
        
        tokens_to_start = tokenizer(text_to_start, return_tensors="pt").input_ids.shape[-1]
        tokens_to_end = tokenizer(text_to_end, return_tensors="pt").input_ids.shape[-1]
        
        # Ensure spans are within bounds
        tokens_to_start = min(tokens_to_start, input_ids.shape[-1])
        tokens_to_end = min(tokens_to_end, input_ids.shape[-1])
        
        if tokens_to_start < tokens_to_end:
            token_spans.append((tokens_to_start, tokens_to_end))
    
    # Run model with cache
    with torch.no_grad():
        logits, cache = model.run_with_cache(input_ids, return_type="logits")
    
    # Compute PKS for each span
    span_pks_scores = []
    for span_idx, (token_start, token_end) in enumerate(token_spans):
        layer_scores = {}
        for layer_idx in range(model.cfg.n_layers):
            x_mid = cache[f"blocks.{layer_idx}.hook_resid_mid"][0, token_start:token_end, :]
            x_post = cache[f"blocks.{layer_idx}.hook_resid_post"][0, token_start:token_end, :]
            
            if x_mid.shape[0] > 0:
                pks = calculate_js_divergence(x_mid, x_post, model.W_U)
            else:
                pks = 0.0
            layer_scores[f"layer_{layer_idx}"] = pks
        
        # Get hallucination label from original scores
        halluc_label = example['scores'][span_idx]['hallucination_label'] if span_idx < len(example['scores']) else 0
        
        span_pks_scores.append({
            'span_idx': span_idx,
            'token_span': (token_start, token_end),
            'pks_by_layer': layer_scores,
            'total_pks': sum(layer_scores.values()),
            'later_layer_pks': sum([v for k, v in layer_scores.items() if int(k.split('_')[1]) >= model.cfg.n_layers // 2]),
            'hallucination_label': halluc_label
        })
    
    # Clean up
    del cache, logits
    gc.collect()
    torch.cuda.empty_cache()
    
    return span_pks_scores

print("Testing span-level PKS computation on trial example 1...")
span_scores = compute_span_pks(trial_examples[0], new_model, tokenizer, device)

print(f"\nSpan-level PKS scores for {trial_examples[0]['id']}:")
for s in span_scores:
    print(f"  Span {s['span_idx']}: Label={s['hallucination_label']}, Total PKS={s['total_pks']:.2f}, Later-layer PKS={s['later_layer_pks']:.2f}")

Testing span-level PKS computation on trial example 1...



Span-level PKS scores for finqa_6345:
  Span 0: Label=0, Total PKS=557.53, Later-layer PKS=152.23
  Span 1: Label=0, Total PKS=569.98, Later-layer PKS=171.53
  Span 2: Label=1, Total PKS=625.76, Later-layer PKS=189.68
  Span 3: Label=1, Total PKS=546.87, Later-layer PKS=136.54
  Span 4: Label=0, Total PKS=358.71, Later-layer PKS=92.41


In [18]:
# Compute span-level PKS for all 3 trial examples
all_span_scores = []
for i, trial_ex in enumerate(trial_examples):
    print(f"Processing trial example {i+1}: {trial_ex['id']}...")
    span_scores = compute_span_pks(trial_ex, new_model, tokenizer, device)
    all_span_scores.extend(span_scores)

print(f"\nTotal spans analyzed: {len(all_span_scores)}")

# Analyze correlation between PKS and hallucination
halluc_pks = [s['later_layer_pks'] for s in all_span_scores if s['hallucination_label'] == 1]
truth_pks = [s['later_layer_pks'] for s in all_span_scores if s['hallucination_label'] == 0]

print(f"\nLater-layer PKS statistics:")
print(f"  Hallucinated spans (n={len(halluc_pks)}): mean={np.mean(halluc_pks):.2f}, std={np.std(halluc_pks):.2f}")
print(f"  Truthful spans (n={len(truth_pks)}): mean={np.mean(truth_pks):.2f}, std={np.std(truth_pks):.2f}")

# Check if hallucinated spans have higher PKS (expected from original finding)
if np.mean(halluc_pks) > np.mean(truth_pks):
    print(f"\n✓ Hallucinated spans have HIGHER later-layer PKS (consistent with original finding)")
    gt1_result = "PASS"
else:
    print(f"\n✗ Hallucinated spans have LOWER later-layer PKS (inconsistent with original finding)")
    gt1_result = "FAIL"

Processing trial example 1: finqa_6345...


Processing trial example 2: finqa_6634...


Processing trial example 3: finqa_6587...



Total spans analyzed: 13

Later-layer PKS statistics:
  Hallucinated spans (n=5): mean=157.93, std=66.59
  Truthful spans (n=8): mean=185.67, std=81.53

✗ Hallucinated spans have LOWER later-layer PKS (inconsistent with original finding)


In [19]:
# Let's also check the total PKS and per-layer patterns
total_halluc_pks = [s['total_pks'] for s in all_span_scores if s['hallucination_label'] == 1]
total_truth_pks = [s['total_pks'] for s in all_span_scores if s['hallucination_label'] == 0]

print("Total PKS statistics:")
print(f"  Hallucinated spans (n={len(total_halluc_pks)}): mean={np.mean(total_halluc_pks):.2f}, std={np.std(total_halluc_pks):.2f}")
print(f"  Truthful spans (n={len(total_truth_pks)}): mean={np.mean(total_truth_pks):.2f}, std={np.std(total_truth_pks):.2f}")

# Analyze per-layer patterns
print("\n\nPer-layer PKS analysis (hallucinated vs truthful):")
for layer_idx in [0, 5, 10, 15, 20, 23]:
    layer_key = f"layer_{layer_idx}"
    halluc_layer = [s['pks_by_layer'][layer_key] for s in all_span_scores if s['hallucination_label'] == 1]
    truth_layer = [s['pks_by_layer'][layer_key] for s in all_span_scores if s['hallucination_label'] == 0]
    diff = np.mean(halluc_layer) - np.mean(truth_layer)
    direction = ">" if diff > 0 else "<"
    print(f"  Layer {layer_idx}: Halluc={np.mean(halluc_layer):.2f}, Truth={np.mean(truth_layer):.2f} (Halluc {direction} Truth by {abs(diff):.2f})")

Total PKS statistics:
  Hallucinated spans (n=5): mean=575.83, std=288.71
  Truthful spans (n=8): mean=616.62, std=224.36


Per-layer PKS analysis (hallucinated vs truthful):
  Layer 0: Halluc=264.00, Truth=278.54 (Halluc < Truth by 14.54)
  Layer 5: Halluc=15.91, Truth=12.93 (Halluc > Truth by 2.98)
  Layer 10: Halluc=13.95, Truth=13.81 (Halluc > Truth by 0.14)
  Layer 15: Halluc=9.80, Truth=12.67 (Halluc < Truth by 2.86)
  Layer 20: Halluc=15.30, Truth=22.02 (Halluc < Truth by 6.72)
  Layer 23: Halluc=18.09, Truth=21.86 (Halluc < Truth by 3.76)
